# PaddleHub之《青春有你2》进行二分类

## 一、任务简介

图像分类是计算机视觉的重要领域，它的目标是将图像分类到预定义的标签。近期，许多研究者提出很多不同种类的神经网络，并且极大的提升了分类算法的性能。本文以自己创建的数据集：青春有你2中选手识别为例子，介绍如何使用PaddleHub进行图像分类任务。

<div  align="center">   
<img src="https://ai-studio-static-online.cdn.bcebos.com/dbaf5ba718f749f0836c342fa67f6d7954cb89f3796b4c8b981a03a1635f2fbe" width = "350" height = "250" align=center />
</div>


In [1]:
#CPU环境启动请务必执行该指令
%set_env CPU_NUM=1 

env: CPU_NUM=1


In [2]:
#安装paddlehup
!pip install paddlehub==1.6.0 -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## 二、任务实践
### Step1、基础工作

加载数据文件

导入python包

In [3]:
!unzip -o /home/aistudio/data/data30796/data.zip

Archive:  /home/aistudio/data/data30796/data.zip
  inflating: data/validate_list.txt  
  inflating: __MACOSX/data/._validate_list.txt  
  inflating: data/.DS_Store          
  inflating: __MACOSX/data/._.DS_Store  
   creating: data/xu/
  inflating: __MACOSX/data/._xu      
  inflating: data/train_list.txt     
  inflating: __MACOSX/data/._train_list.txt  
   creating: data/yu/
  inflating: __MACOSX/data/._yu      
  inflating: data/label_list.txt     
  inflating: __MACOSX/data/._label_list.txt  
  inflating: data/test_list.txt      
  inflating: __MACOSX/data/._test_list.txt  
  inflating: data/xu/.DS_Store       
  inflating: __MACOSX/data/xu/._.DS_Store  
  inflating: data/xu/8.jpg           
  inflating: __MACOSX/data/xu/._8.jpg  
  inflating: data/xu/9.jpg           
  inflating: __MACOSX/data/xu/._9.jpg  
  inflating: data/xu/14.jpg          
  inflating: __MACOSX/data/xu/._14.jpg  
  inflating: data/xu/15.jpg          
  inflating: __MACOSX/data/xu/._15.jpg  
  inflating: data/

In [4]:
import paddlehub as hub

### Step2、加载预训练模型

接下来我们要在PaddleHub中选择合适的预训练模型来Finetune，由于是图像分类任务，因此我们使用经典的ResNet-50作为预训练模型。PaddleHub提供了丰富的图像分类预训练模型，包括了最新的神经网络架构搜索类的PNASNet，我们推荐您尝试不同的预训练模型来获得更好的性能。

In [5]:
module = hub.Module(name="resnet_v2_50_imagenet")

[2021-03-08 16:50:09,220] [    INFO] - Installing resnet_v2_50_imagenet module


[==================================================] 100.00%
Uncompress /home/aistudio/.paddlehub/tmp/tmpq8so69zk/resnet_v2_50_imagenet
[==================================================] 100.00%


[2021-03-08 16:50:16,483] [    INFO] - Successfully installed resnet_v2_50_imagenet-1.0.1


### Step3、数据准备

接着需要加载图片数据集。我们使用自定义的数据进行体验，请查看[适配自定义数据](https://github.com/PaddlePaddle/PaddleHub/wiki/PaddleHub适配自定义数据完成FineTune)

In [6]:
from paddlehub.dataset.base_cv_dataset import BaseCVDataset
   
class DemoDataset(BaseCVDataset):	
   def __init__(self):	
       # 数据集存放位置
       
       self.dataset_dir = "data"
       super(DemoDataset, self).__init__(
           base_path=self.dataset_dir,
           train_list_file="train_list.txt",
           validate_list_file="validate_list.txt",
           test_list_file="test_list.txt",
           #predict_file="predict_list.txt",
           label_list_file="label_list.txt",
           )
dataset = DemoDataset()

In [7]:
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg
img = mpimg.imread('data/xu/1.jpg') 
img1 = mpimg.imread('data/yu/1.jpg') 
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.imshow(img)
plt.axis('off') 
plt.subplot(1,2,2)
plt.imshow(img1)
plt.axis('off') 
plt.show()
print("                许佳琪                                     虞书欣")

2021-03-08 16:50:16,859-INFO: font search path ['/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/ttf', '/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/afm', '/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/mpl-data/fonts/pdfcorefonts']
2021-03-08 16:50:17,212-INFO: generated new fontManager


<Figure size 1000x1000 with 2 Axes>

                许佳琪                                     虞书欣


### Step4、生成数据读取器

接着生成一个图像分类的reader，reader负责将dataset的数据进行预处理，接着以特定格式组织并输入给模型进行训练。

当我们生成一个图像分类的reader时，需要指定输入图片的大小

In [21]:
data_reader = hub.reader.ImageClassificationReader(
    image_width=module.get_expected_image_width(),
    image_height=module.get_expected_image_height(),
    images_mean=module.get_pretrained_images_mean(),
    images_std=module.get_pretrained_images_std(),
    dataset=dataset)

[2021-03-08 16:50:17,521] [    INFO] - Dataset label map = {'许佳琪': 0, '虞书欣': 1}


### Step5、配置策略
在进行Finetune前，我们可以设置一些运行时的配置，例如如下代码中的配置，表示：

* `use_cuda`：设置为False表示使用CPU进行训练。如果您本机支持GPU，且安装的是GPU版本的PaddlePaddle，我们建议您将这个选项设置为True；

* `epoch`：迭代轮数；

* `batch_size`：每次训练的时候，给模型输入的每批数据大小为32，模型训练时能够并行处理批数据，因此batch_size越大，训练的效率越高，但是同时带来了内存的负荷，过大的batch_size可能导致内存不足而无法训练，因此选择一个合适的batch_size是很重要的一步；

* `log_interval`：每隔10 step打印一次训练日志；

* `eval_interval`：每隔50 step在验证集上进行一次性能评估；

* `checkpoint_dir`：将训练的参数和数据保存到cv_finetune_turtorial_demo目录中；

* `strategy`：使用DefaultFinetuneStrategy策略进行finetune；

更多运行配置，请查看[RunConfig](https://github.com/PaddlePaddle/PaddleHub/wiki/PaddleHub-API:-RunConfig)

同时PaddleHub提供了许多优化策略，如`AdamWeightDecayStrategy`、`ULMFiTStrategy`、`DefaultFinetuneStrategy`等，详细信息参见[策略](https://github.com/PaddlePaddle/PaddleHub/wiki/PaddleHub-API:-Strategy)

In [22]:
config = hub.RunConfig(
    use_cuda=True,                              #是否使用GPU训练，默认为False；
    num_epoch=3,                                #Fine-tune的轮数；
    checkpoint_dir="cv_finetune_turtorial_demo",#模型checkpoint保存路径, 若用户没有指定，程序会自动生成；
    batch_size=3,                              #训练的批大小，如果使用GPU，请根据实际情况调整batch_size；
    eval_interval=10,                           #模型评估的间隔，默认每100个step评估一次验证集；
    strategy=hub.finetune.strategy.DefaultFinetuneStrategy())  #Fine-tune优化策略；

[2021-03-08 16:50:17,527] [    INFO] - Checkpoint dir: cv_finetune_turtorial_demo


### Step6、组建Finetune Task
有了合适的预训练模型和准备要迁移的数据集后，我们开始组建一个Task。

由于该数据设置是一个二分类的任务，而我们下载的分类module是在ImageNet数据集上训练的千分类模型，所以我们需要对模型进行简单的微调，把模型改造为一个二分类模型：

1. 获取module的上下文环境，包括输入和输出的变量，以及Paddle Program；
2. 从输出变量中找到特征图提取层feature_map；
3. 在feature_map后面接入一个全连接层，生成Task；

In [23]:
input_dict, output_dict, program = module.context(trainable=True)
img = input_dict["image"]
feature_map = output_dict["feature_map"]
feed_list = [img.name]

task = hub.ImageClassifierTask(
    data_reader=data_reader,
    feed_list=feed_list,
    feature=feature_map,
    num_classes=dataset.num_labels,
    config=config)

[2021-03-08 16:50:17,591] [    INFO] - 267 pretrained paramaters loaded by PaddleHub


### Step5、开始Finetune

我们选择`finetune_and_eval`接口来进行模型训练，这个接口在finetune的过程中，会周期性的进行模型效果的评估，以便我们了解整个训练过程的性能变化。

In [24]:
run_states = task.finetune_and_eval()

[2021-03-08 16:50:18,758] [    INFO] - Strategy with slanted triangle learning rate, L2 regularization, 
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/executor.py:811: UserWarning: There are no operators in the program to be executed. If you pass Program manually, please use fluid.program_guard to ensure the current Program is being used.
  warnings.warn(error_info)
[2021-03-08 16:50:21,194] [    INFO] - Try loading checkpoint from cv_finetune_turtorial_demo/ckpt.meta
[2021-03-08 16:50:21,195] [    INFO] - PaddleHub model checkpoint not found, start from scratch...
[2021-03-08 16:50:21,245] [    INFO] - PaddleHub finetune start
[2021-03-08 16:50:24,430] [   TRAIN] - step 10 / 30: loss=0.72682 acc=0.60000 [step/sec: 3.63]
[2021-03-08 16:50:24,431] [    INFO] - Evaluation on dev dataset start
share_vars_from is set, scope is ignored.
[2021-03-08 16:50:25,115] [    EVAL] - [dev dataset evaluation result] loss=0.56531 acc=0.50000 [step/sec: 23.67]
[2021-03

### Step6、预测

当Finetune完成后，我们使用模型来进行预测，先通过以下命令来获取测试的图片

In [32]:
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.image as mpimg

data = ["data/xu/16.jpg","data/yu/16.jpg"]

label_map = dataset.label_dict()
index = 0
run_states = task.predict(data=data)
results = [run_state.run_results for run_state in run_states]

for batch_result in results:
    print(batch_result)
    batch_result = np.argmax(batch_result, axis=2)[0]
    print(batch_result)
    for result in batch_result:
        index += 1
        result = label_map[result]
        print("input %i is %s, and the predict result is %s" %
              (index, data[index - 1], result))

img = mpimg.imread(data[0]) 
img1 = mpimg.imread(data[1]) 
plt.figure(figsize=(10,10))
plt.subplot(1,2,1)
plt.imshow(img)
plt.axis('off') 
plt.subplot(1,2,2)
plt.imshow(img1)
plt.axis('off') 
plt.show()
print("             input1 许佳琪                             input2 虞书欣")